In [2]:
%config IPCompleter.use_jedi = False
%load_ext autoreload
%autoreload 3
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
from threading import Thread

import h5py
import hdf5storage # conda install hdf5storage
import ipywidgets as widgets


# # required to enable non-blocking interaction:
%gui qt5

from pyphocorehelpers.Filesystem.HDF5.hdf5_file_helpers import h5dump, HDF5_Helper
from pyphocorehelpers.Filesystem.HDF5.silx_hdf5_viewer import hdf5_tree_view_window

from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, get_file_metadata, FileList
from pyphocorehelpers.print_helpers import debug_dump_object_member_shapes, print_value_overview_only, pprint, debug_print, print_keys_if_possible
from pyphoplacecellanalysis.PhoPositionalData.load_exported import import_mat_file
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, extract_spike_timeseries
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import _test_save_pipeline_data_to_h5, get_h5_data_keys, save_some_pipeline_data_to_h5, load_pipeline_data_from_h5  #ExportHelpers

from pyphoplacecellanalysis.GUI.PyQtPlot.Params.MatFileBrowser.MatFileBrowser import MatFileBrowser

## Drastically improve Python Tracebacks on exceptions:
# from rich.traceback import install
# install()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# RoyMaze1:

mat_import_parent_path = Path(r'R:\data\RoyMaze1')
# mat_import_parent_path = Path(r'C:\Share\data\RoyMaze1') # Old one
# mat_import_file = r'C:\Share\data\RoyMaze1\ExportedData.mat'

found_files = MatFileBrowser.discover_mat_files(mat_import_parent_path, recursive=True)
found_files

In [ ]:
# from param import define, field, Factory
# class FilesCollection:




In [26]:
search_parent_path = Path(r'W:\\Data\\Kdiba')

found_any_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='*.pkl', recursive=True) # any pickle, anywhere
found_other_outputs_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='output/*.pkl', recursive=True) # any pickle in the `output/` folder


## Secondary types:
found_autoversioned_session_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='*-loadedSessPickle.pkl', recursive=True)


## Core desired types:
found_default_session_pickle_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='loadedSessPickle.pkl', recursive=True)
found_global_computation_results_files: list[Path] = discover_data_files(search_parent_path, glob_pattern='output/global_computation_results.pkl', recursive=True) # 'output/global_computation_results.pkl'

all_core_desired_files = found_default_session_pickle_files + found_global_computation_results_files

# found_files: list[Path] = found_default_session_pickle_files + found_global_computation_results_files + found_autoversioned_session_pickle_files
# found_files

# non_primary_desired_files = to_set(found_any_pickle_files)[0] - to_set(all_core_desired_files)[0]
# non_primary_desired_files = from_set(to_set(found_any_pickle_files)[0] - to_set(found_default_session_pickle_files + found_global_computation_results_files)[0])
# non_primary_desired_files = FileList.from_set(FileList.to_set(found_any_pickle_files) - FileList.to_set(found_default_session_pickle_files + found_global_computation_results_files))
non_primary_desired_files = FileList.subtract(found_any_pickle_files, all_core_desired_files)
non_primary_desired_files

# all_core_desired_files
non_primary_desired_files

[WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-08_14-26-15/out_all_ripple_results.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-07_16-40-19/ripple_detector.pkl'),
 WindowsPath('W:/Data/Kdiba/vvp01/one/2006-4-18_13-6-1/ripple_df.pkl'),
 WindowsPath('W:/Data/Kdiba/vvp01/two/2006-4-11_16-2-46/ripple_detector.pkl'),
 WindowsPath('W:/Data/Kdiba/vvp01/two/2006-4-21_11-19-2/ripple_detector.pkl'),
 WindowsPath('W:/Data/Kdiba/pin01/one/11-02_17-46-44/output/leave_one_out_results_long.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-09_22-24-40/ripple_detector.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-07_16-40-19/output/leave_one_out_results_short.pkl'),
 WindowsPath('W:/Data/Kdiba/vvp01/one/2006-4-19_16-48-9/ripple_df.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-13_14-42-6/ripple_detector.pkl'),
 WindowsPath('W:/Data/Kdiba/vvp01/one/2006-4-28_16-48-29/ripple_detector.pkl'),
 WindowsPath('W:/Data/Kdiba/pin01/one/11-02_17-46-44/output/leave_one_out_results_short.pkl'),
 Wi

- https://realpython.com/inherit-python-list/
- https://stackoverflow.com/questions/8180014/how-to-subclass-python-list-without-type-problems/8180073#8180073
- https://docs.python.org/3/reference/datamodel.html#emulating-container-types
- https://docs.python.org/3/library/collections.html#collections.UserList
- https://stackoverflow.com/questions/50470557/pythonic-way-of-subclassing-a-list
- https://vetude.com/part-9-list-in-python-an-introduction-to-subclassing-a-list/
- https://datagy.io/python-collections-userlist/
- https://sentry.io/answers/what-are-Python-metaclasses/
- https://stackoverflow.com/questions/25328448/should-i-subclass-list-or-create-class-with-list-as-attribute- 

In [25]:
split_on_nearest_data_directory = lambda a_path: next(((tuple(a_path.relative_to(parent).parts[:-1]), tuple(a_path.relative_to(parent).parts[-1:])) for parent in a_path.parents if parent.name.lower() == 'data'), None)

# my_list = ['path', 'to', 'some', 'Data', 'directory', 'file.txt']
my_list = a_path.parts # ('W:\\', 'Data', 'Kdiba', 'gor01', 'one', '2006-6-07_11-26-53', 'loadedSessPickle.pkl')
search_element = 'Data'

# matches = [(i, elem) for i, elem in enumerate(reversed(my_list)) if elem.lower() == search_element.lower()]

try:
    index = len(my_list) - my_list[::-1].index('Data') - 1
    print("Index of the first element matching 'Data':", index)
    list_before = my_list[:index]
    list_after = my_list[index:]
    print("List before index:", list_before)
    print("List after index:", list_after)
    data_folder_relative_path = Path(*list_after)
    print(f'data_folder_relative_path: {data_folder_relative_path}')
except ValueError:
    print("No element matching 'Data' found in the list.")
    
a_path = all_core_desired_files[0]
(str(a_path.parent.as_posix()), data_folder_relative_path, a_path.name)
# a_path.as_posix()


# split_parts = split_on_nearest_data_directory(a_path)
# if split_parts:
#     before_data, after_data = split_parts
#     print('Before Data:', before_data)
#     print('After Data:', after_data)
# else:
#     print("No 'Data' or 'data' parent directory found in the path.")
    
# a_path.parts.reversed().index('Data')
# print(a_path.parts) # ('W:\\', 'Data', 'Kdiba', 'gor01', 'one', '2006-6-07_11-26-53', 'loadedSessPickle.pkl')
# index

Index of the first element matching 'Data': 1
List before index: ('W:\\',)
List after index: ('Data', 'Kdiba', 'gor01', 'one', '2006-6-07_11-26-53', 'loadedSessPickle.pkl')
data_folder_relative_path: Data\Kdiba\gor01\one\2006-6-07_11-26-53\loadedSessPickle.pkl


('W:/Data/Kdiba/gor01/one/2006-6-07_11-26-53',
 WindowsPath('Data/Kdiba/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl'),
 'loadedSessPickle.pkl')

In [6]:
# pd.DataFrame({'path': found_files})

all_core_desired_files_df = get_file_metadata(all_core_desired_files)
all_core_desired_files_df['path']


all_core_desired_files_df['filename'] = 

0     W:\Data\Kdiba\gor01\one\2006-6-07_11-26-53\loa...
1     W:\Data\Kdiba\gor01\one\2006-6-08_14-26-15\loa...
2     W:\Data\Kdiba\gor01\one\2006-6-09_1-22-43\load...
3     W:\Data\Kdiba\gor01\one\2006-6-09_3-23-37\load...
4     W:\Data\Kdiba\gor01\one\2006-6-12_15-55-31\loa...
                            ...                        
56    W:\Data\Kdiba\vvp01\one\2006-4-19_13-34-40\out...
57    W:\Data\Kdiba\vvp01\two\2006-4-09_16-40-54\out...
58    W:\Data\Kdiba\vvp01\two\2006-4-10_12-58-3\outp...
59    W:\Data\Kdiba\vvp01\two\2006-4-12_15-25-59\out...
60    W:\Data\Kdiba\vvp01\two\2006-4-16_14-49-24\out...
Name: path, Length: 61, dtype: object

In [ ]:
found_file_variables_dict = MatFileBrowser.whos_mat_files(found_files)
print(found_file_variables_dict)

# Silx hdf5 Widgets

In [ ]:
non_hdf5_files = [Path(r'R:/data/RoyMaze1/ExportedData/extrasAnalysis.mat'), Path('R:/data/RoyMaze1/ExportedData/RippleIndexedSpikes.mat')] # , Path(r'R:/data/RoyMaze1/ExportedData/positionAnalysis.mat')
hdf5_found_files = [str(found_file) for found_file in found_files if found_file not in non_hdf5_files]
print(f'hdf5_found_files: {hdf5_found_files}')
win = hdf5_tree_view_window(hdf5_found_files)
win

# Overflow (Not needed, Slix HDF5 widget worked to display/browse the data)

In [ ]:
# Import the positions
t,x,y,speeds,dt,dx,dy = perform_import_positions(mat_import_parent_path=mat_import_parent_path)
# Import the spikes
spike_matrix, spike_cells, num_cells, spike_list, spike_positions_list = perform_import_spikes(t, x, y, mat_import_parent_path=mat_import_parent_path)

ripple_times, ripple_peak_times, ripple_peak_values, behavioral_periods, behavioral_epochs = perform_import_extras(mat_import_parent_path=mat_import_parent_path)

# behavioral_periods = all_results_data['active_processing/behavioral_periods_table']
# print('spike_matrix: {}, spike_cells: {}'.format(np.shape(spike_matrix), np.shape(spike_cells)))
num_periods = np.shape(behavioral_periods)[0]
print('num_periods: {}'.format(num_periods))


# creating the dataset
#date = [i for i in range(2000, 2021)]
 
# Create a figure and a set of subplots
fig, ax = plt.subplots(figsize=(10, 6))
 
# Creating the scatter plot
#ax.scatter(date, value)
 
# Highlighting for a certain period of time
num_ripples = np.shape(ripple_times)[0]
for i in np.arange(num_ripples):
    ax.axvspan(ripple_times[i,0], ripple_times[i,1], alpha=0.3, color="green")

plt.show()

In [ ]:
import silx.gui.hdf5
treeview = silx.gui.hdf5.Hdf5TreeView()
model = treeview.findHdf5TreeModel()
# Avoid the user to drop file in the widget
model.setFileDropEnabled(False)
# Allow the user to reorder files with drag-and-drop
model.setFileMoveEnabled(True)
header = treeview.header()
# Select displayed columns
column_ids = [treeview.findHdf5TreeModel().NAME_COLUMN]
header.setSections(column_ids)
# Do not allow the user to custom visible columns
header.setEnableHideColumnsPopup(False)

## MatFileBrowser.build_browsing_gui(...) Parameter Tree GUI (WORKING, NOT USED)

In [ ]:
tree, app = MatFileBrowser.build_browsing_gui(found_file_variables_dict)
tree.show()

## HDF5_Helper Recurrsion Testing (NOT FINISHED, NOT WORKING, NOT USED)

In [ ]:
# found_file = Path(r'R:/data/RoyMaze1/analysesResults_30-Sep-2021/allEnv.mat')
# found_file = Path(r'R:/data/RoyMaze1/ExportedData/spikesAnalysis.mat')
# found_file = Path(r'R:\data\RoyMaze1\ExportedData\extrasAnalysis.mat')
found_file = Path(r'R:/data/RoyMaze1/ExportedData/spikesTable.mat')

In [ ]:
# import h5py
helper = HDF5_Helper(path=found_file, group='active_processing')
helper.output_tree

In [ ]:
# helper.perform_enumerate(debug_print=True)
helper.perform_enumerate(debug_print=False)
helper.output_tree

In [ ]:
found_file_variables_dict[found_file]

In [ ]:
h5dump(found_file, group='active_processing', enable_print_attributes=False)

## h5py 'visit_item' testing (WORKING, NOT USED):

In [ ]:
def pho_visit_item_fn(name, obj):
    # Recursively visit all objects in this group and subgroups. Like Group.visit(), except your callable should have the signature:
    # callable(name, object) -> None or return value
    # In this case object will be a Group or Dataset instance.
    # if name not in ignored_keys:        
    if '#' in name:
        return None
    else:
        print(f'name: {name}, obj: {obj}')
        # if type(obj) in [h5py._hl.group.Group, h5py._hl.files.File]:
        #     for key in obj.keys():
        #         print(sep,'-',key,':',obj[key])
        #         descend_obj(obj[key],sep=sep+'\t')
        # elif type(obj)==h5py._hl.dataset.Dataset:
        #     for key in obj.attrs.keys():
        #         print(sep+'\t','-',key,':',obj.attrs[key])
            
    
def find_foo(name):
    """ Find first object with 'foo' anywhere in the name """
    if 'foo' in name:
        return name

with h5py.File(found_file,'r') as f:
    print(f'file:{found_file}: {f}')
    # print(f.visititems)
    _out = f.visititems(pho_visit_item_fn)
    # group.visit(find_foo)

    # f.attrs
    # print_value_overview_only(f)
    # # found_data_dict = {a_key:meta_info_dict for a_key in list(f.keys()) if a_key not in ignored_keys} ## filter the ignored keys
    # # found_keys = [a_key for a_key in list(f.keys()) if a_key not in ignored_keys] ## filter the ignored keys
    # f.name

## QFileSystemModel, QTreeView: Fresh Implementation (NOT USED)

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QFileSystemModel, QTreeView, QWidget, QVBoxLayout
from PyQt5.QtGui import QIcon

class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'PyQt5 file system view - pythonspot.com'
        self.left = 10
        self.top = 10
        self.width = 640
        self.height = 480
        self.initUI()
    
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        self.model = QFileSystemModel()
        self.model.setRootPath(r'E:\data')
        self.tree = QTreeView()
        self.tree.setModel(self.model)
        
        self.tree.setAnimated(False)
        self.tree.setIndentation(20)
        self.tree.setSortingEnabled(True)
        
        self.tree.setWindowTitle("Dir View")
        self.tree.resize(640, 480)
        
        windowLayout = QVBoxLayout()
        windowLayout.addWidget(self.tree)
        self.setLayout(windowLayout)
        
        self.show()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())